# Lab: Machine Learning and BigQuery GIS
This lab will teach you how to perform a clustering analysis in BigQuery GIS using Python and Jupyter notebooks.

[BigQuery](https://cloud.google.com/bigquery/docs) is a serverless data warehouse solution on Google Cloud Platform (GCP) that allows users to interact with their data using standard ANSI SQL.

Using the BigQuery Python API, BigQuery databases can be directly accessed via Python and/or Jupyter notebooks.

BigQuery GIS allows point, line or polygon geospatial data to be stored as a special `GEOGRAPHY` data type within a BigQuery table.  This `GEOGRAPHY` datatype is stored as a [well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) object. 

## Getting Started
In order to connect to BigQuery in Python, you need to install the BigQuery Python client library and the BigQuery Storage API.  You will also need to set up the notebook to use the "magic" command, which we will be using to access BigQuery throughout this lab.  

In addition, you will need to install the `pandas` package if you don't have it already installed.

Note: If you are running this notebook in a GCP environment, these packages should be pre-installed.

In [ ]:
%pip install 'google-cloud-bigquery'
%pip install 'google-cloud-bigquery-storage'
%pip install 'pyarrow'
%pip install 'pandas'

%load_ext google.cloud.bigquery

## DBScan: Density-Based Clustering
[Clustering](https://en.wikipedia.org/wiki/Cluster_analysis) is a machine learning methodology that groups data based on common attributes. This is considered an unsupervised algorithm because there is no "right" answer that you are trying to predict. The clusters are discovered in the course of running the analysis, with no guidance on how they should form beyond the algorithm itself. 

The built-in algorithm for clustering in BigQuery GIS is called [DBScan](https://en.wikipedia.org/wiki/DBSCAN), which is a density-based clustering algorithm (the DB stands for Density Based). This means that while data points in high-density areas get clustered together, data points in low-density areas are not put into any cluster but instead are classified as "noise."

One nice thing about the DBScan algorithm is that you don't need to specify the number of clusters in advance. This is in contrast to [K-means clustering](https://en.wikipedia.org/wiki/K-means_clustering), where K is the number of clusters and is a required input for the analysis. 

## Creating Clusters
For this analysis, we will be using the San Francisco street tree data in Google's [public datasets](https://cloud.google.com/public-datasets) available within BigQuery.

The first step is to assign each tree in the dataset to either a cluster or "noise."  This will be done using the [ST_CLUSTERDBSCAN](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_clusterdbscan) function within BigQuery GIS.

This function takes three parameters as its arguments:
1. A column of `GEOGRAPHY` values to be clustered.
2. The radius around a core value for each cluster (measured in meters).
3. The minimum number of `GEOGRAPHY` items within a cluster.

For our example, the `GEOGRAPHY` column will be our list of street trees.  The radius will be 100 meters and the minimum number of items within each cluster will be 10 trees.

Let's go ahead and create a query that assigns each tree to a cluster.  We can directly write ANSI SQL to query BigQuery tables by using the `%%bigquery` [magic command](https://googleapis.dev/python/bigquery/latest/magics.html).

We're going to save our results into a `pandas` dataframe called "tree_clusters" by adding that label after the magic command.

In [1]:
%%bigquery tree_clusters
select tree_id, species, legal_status, care_taker, ST_GEOGPOINT(longitude,latitude) tree_geom,
ST_CLUSTERDBSCAN(ST_GEOGPOINT(longitude,latitude),100,10) over () cluster_id
FROM `bigquery-public-data.san_francisco_trees.street_trees`
WHERE latitude is not null;

Notice that the "street_trees" table does not have a native `GEOGRAPHY` field, so we need to create one using the [ST_GEOGPOINT](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_geogpoint) constructor function from the latitude/longitude coordinate values.  We are also excluding trees that do not have coordinate values from our clustering analysis.

We can view the first 15 rows of the `pandas` dataframe created from our SQL query results by using the [head](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) method.

In [2]:
#Display the head of the tree_clusters dataframe
tree_clusters.head(15)

,tree_id,species,legal_status,care_taker,tree_geom,cluster_id
0,34336,Jacaranda mimosifolia :: Jacaranda,Permitted Site,Private,POINT(-122.402814623369 37.7500487659499),0.0
1,84716,Washingtonia robusta :: Mexican Fan Palm,Permitted Site,Private,POINT(-122.437947368595 37.7690249659747),0.0
2,34308,Jacaranda mimosifolia :: Jacaranda,Permitted Site,Private,POINT(-122.402814623369 37.7500487659499),0.0
3,45378,Tree(s) ::,Permitted Site,Private,POINT(-122.480907237859 37.7928819736725),1.0
4,45381,Lophostemon confertus :: Brisbane Box,Permitted Site,Private,POINT(-122.480907237859 37.7928819736725),1.0
5,34329,Jacaranda mimosifolia :: Jacaranda,Permitted Site,Private,POINT(-122.402814623369 37.7500487659499),0.0
6,38764,Tristaniopsis laurina :: Swamp Myrtle,Permitted Site,Private,POINT(-122.436517937945 37.7309323858205),0.0
7,34309,Jacaranda mimosifolia :: Jacaranda,Permitted Site,Private,POINT(-122.402814623369 37.7500487659499),0.0
8,45376,Tree(s) ::,Permitted Site,Private,POINT(-122.480907237859 37.7928819736725),1.0
9,66273,Tree(s) ::,Permitted Site,Private,POINT(-122.401695868421 37.7920108636178),0.0


**Try It:**
Can you create clusters with 200 meter radii and a minimum of 15 trees in each cluster?  
*Hint:* Review the required arguments to the `ST_CLUSTERDBSCAN` function.

In [ ]:
%%bigquery tree_clusters_200m
select tree_id, species, legal_status, care_taker, ST_GEOGPOINT(longitude,latitude) tree_geom,
ST_CLUSTERDBSCAN(??,??,??) over () cluster_id
FROM `bigquery-public-data.san_francisco_trees.street_trees`
WHERE latitude is not null;

In [ ]:
#Note: Entering no argument in the "head()" method defaults to the first 5 rows
tree_clusters_200m.head()

## Examining Clusters with `pandas`
We can now use [pandas](https://pandas.pydata.org/) DataFrame methods to do aggregation on our "tree_clusters" DataFrame to count the number of trees in each cluster.

`pandas` is a popular data analysis and manipulate package for Python that is a common tool in the data science toolkit.  If you are unfamiliar with `pandas`, check out [this introduction](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html).  You can also find the full user guide [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html).

While the syntax may be different, many of the functions that can be performed on a `pandas` DataFrame are similar to functions that can be performed in SQL:
* The [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) method groups on the specified column (or list of columns) (compare to SQL's "GROUP BY")
* The [count](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.count.html) method counts rows in each group (compare to SQL's "COUNT(*)")
* The [sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) method allows us to sort descending by count (compare to SQL's "ORDER BY")

In [3]:
tree_clusters[['cluster_id','tree_id']].groupby('cluster_id').count().sort_values(by='tree_id', ascending=False)

,tree_id
cluster_id,
0.0,189299
4.0,524
6.0,129
7.0,107
22.0,103
9.0,93
21.0,86
13.0,55
17.0,48


There are 34 clusters that were created by the DBScan algorithm.

**Try It:**  
Can you count the number of trees in each of your clusters with 200 meter radii and a minimum of 15 trees in each cluster?  How many clusters are created?

In [ ]:
??.groupby(??).count().sort_values(by='tree_id', ascending=False)

## Noise Data Points
As expected for the DBScan algorithm, some trees were not assigned to any cluster.  Instead, they were assigned to "noise" since they were not near enough to a sufficient number of trees to be considered part of a cluster.  

For the "noise" trees, the cluster_id is assigned to "NULL" in SQL or "NaN" in Python.  You can find these "noise" trees by using the [.isna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html) function in `pandas` to filter the "tree_clusters" dataframe.

In [4]:
tree_clusters[tree_clusters['cluster_id'].isna()]

,tree_id,species,legal_status,care_taker,tree_geom,cluster_id
14,34235,Magnolia grandiflora :: Southern Magnolia,Permitted Site,Private,POINT(-122.384568876102 37.7577074184489),NaN
56,65996,Tree(s) ::,Permitted Site,Private,POINT(-122.480532017343 37.7906402729338),NaN
71,66415,Tree(s) ::,Permitted Site,Private,POINT(-122.420954201226 37.808658736106),NaN
80,65084,Tree(s) ::,Permitted Site,Private,POINT(-122.476412847781 37.7979444122084),NaN
109,39330,Melaleuca quinquenervia :: Cajeput,Permitted Site,Private,POINT(-122.384018475054 37.7603812969624),NaN
...,...,...,...,...,...,...
188215,196275,Prunus cerasifera :: Cherry Plum,DPW Maintained,Private,POINT(-122.377627436781 37.729921505667),NaN
188847,176391,Eucalyptus sideroxylon :: Red Ironbark,DPW Maintained,Private,POINT(-122.428352360384 37.7119904737556),NaN
189720,201932,Juniperus chinensis :: Juniper,DPW Maintained,Private,POINT(-122.405672426066 37.7347570640926),NaN
189721,201933,Metrosideros excelsa :: New Zealand Xmas Tree,DPW Maintained,Private,POINT(-122.405683976059 37.7347493344117),NaN


For our clusters with 100-meter radii and a minimum of 10 trees, there were 286 trees assigned to "noise."

**Try It:**  
Can you identify the number of trees assigned to "noise" for your analysis with 200 meter radii and a minimum of 15 trees in each cluster?  
*Hint:* Consider combining the `.isna()` function and the `count()` function. 

In [ ]:
tree_clusters_200m[??].??